* Read Language model tutorial ---> https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275
* Find one english corpus with poetries in the internet (e.g from here) --> https://www.poetryfoundation.org/poems
* You can use whatever corpus you want (e.g. your favorite book)
* Encapsulate LSTM building like MLP from the first task
* Train LSTM as language model on your corpus like in the tutorial
* Also, you need to compare 1-layer and 2-layer LSTMs
* Compare texts, generated by your models

In [0]:
from __future__ import print_function

import keras
from keras.layers import Input, Embedding, LSTM, Dense, Dropout
from keras.utils import np_utils
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import adam, adagrad, adadelta, rmsprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.regularizers import L1L2

import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid, train_test_split

import os
import numpy as np
import pandas as pd


os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Read Data
folder_name = '/content/gdrive/My Drive/Colab Notebooks/'
filename = os.path.join(folder_name, 'task3_corpus.csv')
# filename = 'task3_corpus.csv'
file_type = 'csv'

In [0]:
def read_data(filename, file_type):
    if file_type == 'csv':
        data = pd.read_csv(filename)
        data = data['text']
        
    return data

In [0]:
df = read_data(filename, file_type)

In [0]:
text = '\n'.join([row for row in df])

In [45]:
layers = [( 'LSTM', 150), ('Dropout', 0.2), ('LSTM', 120)]
count = [x for x,_ in layers].count('LSTM')
print(count)


2


In [0]:
class ModelFormer:
    def __init__(self):
        self.x = []
        self.y = []
        self.tokenizer = Tokenizer()
        self.best_model = Sequential()
        self.best_accuracy = 0
        self.best_parameters = {}
        
    def fit_data(self, text):
        self.original_corpus = text
        self.corpus = self.original_corpus.lower().split('\n')
        self.tokenizer.fit_on_texts(self.corpus)
        self.word_count = len(self.tokenizer.word_index) + 1
        input_sequences = []
        for line in self.corpus:
            tokens = self.tokenizer.texts_to_sequences([line])[0]
            for i in range(1, len(tokens)):
                n_grams_sequence = tokens[:i+1]
                input_sequences.append(n_grams_sequence)
        
        input_sequences = self.pad_input_sequences(input_sequences)
        
        x_data, y_data = input_sequences[:,:-1], input_sequences[:,-1]
        y_data = np_utils.to_categorical(y_data, num_classes=self.word_count)
        
        return x_data, y_data
              
    def pad_input_sequences(self,input_sequences):
        max_sequence_length = max([len(sentence) for sentence in input_sequences])
        input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))
        return input_sequences
    
    def fit(self, x_data, y_data , layers= [( 'LSTM', 150), ('Dropout', 0.2), ('LSTM', 120)], activation='tanh', optimizer='adam', lr=0.01, epochs=20):
        self.model = Sequential()
        
        self.x_data = x_data
        self.y_data = y_data
        x_train, x_val, y_train, y_val = train_test_split(self.x_data, self.y_data)
        
        
        self.model.add(Embedding(self.word_count, 15, input_length=len(x_data[0]) ))
        count_lstm_retn_flag = [x for x,_ in layers].count('LSTM') - 1

        for layer,value in layers:
            if layer == 'LSTM':
                if count_lstm_retn_flag:
                    count_lstm_retn_flag -= 1
                    return_sequences = True 
                else:
                    return_sequences = False
                self.model.add(LSTM(value, activation=activation, return_sequences=return_sequences))
            if layer == 'Dropout':
                self.model.add(Dropout(value))
        
        self.model.add(Dense(self.word_count, activation='softmax'))
        if optimizer == 'adam':
            optimizer = adam(lr=lr)
        elif optimizer == 'adadelta':
            optimizer = adadelta(lr=lr)
        elif optimizer == 'rmsprop':
            optimizer = rmsprop(lr=lr)
            
            
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.model.summary()
        
        fit_summary = self.model.fit(x_train, y_train, epochs=epochs, verbose=1, validation_data=(x_val, y_val), batch_size=20)
        if fit_summary.history['acc'][-1] > self.best_accuracy:
            self.best_model = self.model
            self.best_accuracy = fit_summary.history['acc'][-1]
            self.best_parameters = (layers, activation, optimizer, lr, epochs)
        
        return fit_summary
        
        

In [0]:
m = ModelFormer()
X, Y= m.fit_data(text)


In [0]:
x_train , x_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

In [111]:
print(len(x_train), len(y_train), len(x_test), len(y_test))

8847 8847 3792 3792


In [0]:
# Define Grid Search with Parameter Grid

In [0]:
hyperparameters = { 'layers': [[( 'LSTM', 10), ('Dropout', 0.2), ('LSTM', 20), ('Dropout', 0.2) ], [( 'LSTM', 10), ('Dropout', 0.2)]], 
                     'activation': ['tanh', 'sigmoid'],
                     'optimizer' : ['adam', 'adadelta', 'rmsprop'],
                     'lr' : [0.1, 0.01, 0.001],
                     'epochs' : [50]
                   }

In [114]:
combinations = list(ParameterGrid(hyperparameters))
len(combinations)

36

In [0]:
fit_summary_array = []

In [116]:
for combination in combinations:
    print('Current Combination : {}'.format(combination))
    m.fit(x_train, y_train, layers=combination['layers'], activation=combination['activation'], optimizer=combination['optimizer'], lr=combination['lr'], epochs=combination['epochs'])

Current Combination : {'activation': 'tanh', 'epochs': 50, 'layers': [('LSTM', 10), ('Dropout', 0.2), ('LSTM', 20), ('Dropout', 0.2)], 'lr': 0.1, 'optimizer': 'adam'}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 54, 15)            32775     
_________________________________________________________________
lstm_23 (LSTM)               (None, 54, 10)            1040      
_________________________________________________________________
dropout_21 (Dropout)         (None, 54, 10)            0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 20)                2480      
_________________________________________________________________
dropout_22 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 2185)

KeyboardInterrupt: ignored